In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
file = '/content/drive/MyDrive/hamoye/energydata_complete.csv'
energy_file = pd.read_csv(file)

In [ ]:
energy_file.head()

In [ ]:
linear_reg = energy_file[['T2', 'T6']].sample(15, random_state = 2)

In [ ]:
sns.regplot(x='T2', y='T6', data = linear_reg)

In [ ]:
energy_df = energy_file.drop(['date', 'lights'], axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(energy_df), columns=energy_df.columns)
appliances_target = normalised_df['Appliances']

In [ ]:
#split into train and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(energy_df, appliances_target, test_size=0.3, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression #import linear regressiom
linear_model = LinearRegression()
linear_model.fit(x_train, y_train) #fit the model to the training dataset
pred_values = linear_model.predict(x_test) #predictions from the model

In [ ]:
#MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, pred_values)
round(mae, 2)

In [ ]:
#RSS
rss = np.sum(np.square(y_test - pred_values))
round(rss, 2) 

In [ ]:
#RMSE
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, pred_values))
round(rmse, 3)

In [ ]:
#RSquared
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, pred_values)
round(r2_score, 2)

In [ ]:
#Ridge regression
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)


In [ ]:
#Lasso regression
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

In [ ]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df
  
linear_model_weights = get_weights_df(model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
